---
title: Composite Data
abstract: |
    Instead of managing individual variables separately, composite data types enable developers to bundle related data together and access them conveniently. This concept is essential for modeling complex information and reducing redundancy. In this notebook, we will explore ordered sequences including fixed-length arrays and variable-length vectors in C++.
math:
    '\abs': '\left\lvert #1 \right\rvert'
    '\norm': '\left\lVert #1 \right\rVert'
    '\Set': '\left\{ #1 \right\}'
    '\set': '\operatorname{set}'   
    '\mc': '\mathcal{#1}'
    '\M': '\boldsymbol{#1}'
    '\R': '\mathsf{#1}'
    '\RM': '\boldsymbol{\mathsf{#1}}'
    '\op': '\operatorname{#1}'
    '\E': '\op{E}'
    '\d': '\mathrm{\mathstrut d}'
    '\SFM': '\operatorname{SFM}'
    '\utag': '\stackrel{\text{(#1)}}{#2}'
    '\uref': '\text{(#1)}'
    '\minimal': '\operatorname{minimal}'
skip_execution: true
---

In [ ]:
from __init__ import *
%reload_ext divewidgets

In [ ]:
if not input('Load JupyterAI? [Y/n]').lower()=='n':
    %reload_ext jupyter_ai

## Motivation

Composite data refers to structured groupings of multiple values under a single entity, allowing for more organized and efficient data handling.  In this notebook, we will explore ordered sequences including fixed-length arrays and variable-length vectors in C++. Let us begin with some motivating examples.

### Cross Product

Consider computing the [cross product](https://en.wikipedia.org/wiki/Cross_product#Lie_algebra) of two vectors, which has many applications in physics and computer graphics:

::::{prf:definition} cross product

The cross product of two vectors 

$$
\begin{align}
\M{u}&=\begin{bmatrix} u_0 & u_1 & u_2 \end{bmatrix}\\
\M{v}&=\begin{bmatrix} v_0 & v_1 & v_2 \end{bmatrix}
\end{align}
$$ (eq:cross)

in the 3D real vector space $\mathbb{R}^3$ is a vector defined as

$$
\M{u} \times \M{v} := \begin{bmatrix} u_1v_2 - u_2v_1,\ u_2v_0 - u_0v_2,\ u_0v_1 - u_1v_0 \end{bmatrix}.
$$

::::

One way to implement the cross product is to use [C-style fixed-size arrays][carray]:

::::{code} cpp
:label: code_cross_cstyle
:caption: Compute the cross product $\M{u}\times \M{v}$ using [C-style array][carray].
:linenos:
/**
 * @brief Computes the cross product of two 3D vectors.
 *
 * This function calculates the cross product of two input vectors `u` and `v`,
 * each represented as a fixed-size array of 3 doubles. The result is stored in
 * the output array `w`, which must also be of size 3.
 *
 * @param u A constant reference to a 3-element array representing the first vector.
 * @param v A constant reference to a 3-element array representing the second vector.
 * @param w A reference to a 3-element array where the result (cross product) will be stored.
 * @example
 * double w[3];
 * cross((double[]){1,0,0}, (double[]){0,1,0},w); // (double[3]) { 0.0, 0.0, 1.0 }
 */
void cross(const double (&u)[3], const double (&v)[3], double (&w)[3]) {
    w[0] = u[1]*v[2] - u[2]*v[1];
    w[1] = u[2]*v[0] - u[0]*v[2];
    w[2] = u[0]*v[1] - u[1]*v[0];
}
::::

[carray]: https://en.cppreference.com/w/cpp/language/array.html

In [ ]:
%%cpp
void cross(const double (&u)[3], const double (&v)[3], double (&w)[3]) {
    w[0] = u[1]*v[2] - u[2]*v[1];
    w[1] = u[2]*v[0] - u[0]*v[2];
    w[2] = u[0]*v[1] - u[1]*v[0];
}
cross

For instance, to compute $\M{w}=\M{u}\times \M{v}$ with

\begin{align}
\M{u}&=\begin{bmatrix} 1 & 0 & 0\end{bmatrix}\\
\M{v}&=\begin{bmatrix} 0 & 1 & 0\end{bmatrix}:
\end{align}

In [ ]:
%%cpp
double w[3];
cross((double[]){1,0,0}, (double[]){0,1,0},w);
w

::::{exercise}
:label: ex:cross

Interestingly, the resulting cross product is not returned by the function `cross`, but rather, stored in an argument `w` declared outside the function and passed by reference to the function:

```cpp
void cross(..., ..., double (&w)[3]) {
    w[0] = ...
    w[1] = ...
    w[2] = ...
}
```

Why not declare `w` inside the function and return it?

::::

YOUR ANSWER HERE

We'll explore an alternative implementation of the dot product using C++11's fixed-size container [`std::array`][array] from `<array>`, which can be safely and properly returned by a function similar to other basic data types.

[array]: https://en.cppreference.com/w/cpp/container/array.html

### Dot Product

Consider computing the [dot product](https://en.wikipedia.org/wiki/Dot_product) of two vectors, which measures how much one vector points in the direction of another. The dot product has numerous applications beyond physics and computer graphics, including statistics and machine learning.

::::{prf:definition} dot product

The dot product of two vectors 

$$
\begin{align}
\M{u}&=\begin{bmatrix} u_0 & u_1 & \dots & u_{n-1} \end{bmatrix}\\
\M{v}&=\begin{bmatrix} v_0 & v_1 & \dots & v_{n-1} \end{bmatrix}
\end{align}
$$ 

in n-dimensional real vector space $\mathbb{R}^n$ is

$$
\M{u} \cdot \M{v} := \sum_{i=1}^{n} u_i v_i.
$$ (eq:dot)

The length/norm of a vector $\M{v}$ is

$$
\begin{align}
\norm{\M{v}}:= \sqrt{\M{v}\cdot \M{v}}.
\end{align}
$$ (eq:norm)

::::

To implement [](#eq:dot) properly with C-style fixed-size array:

::::{code} cpp
:label: code_dot_cstyle
:caption: Compute the dot product $\M{u}\cdot \M{v}$ using C-style array.
:linenos:
/**
 * @brief Computes the dot product of two arrays of doubles.
 *
 * This function calculates the dot product of two fixed-size arrays `u` and `v`.
 * It assumes both arrays have the same length `n`.
 *
 * @param u A reference to an array of doubles.
 * @param v A reference to an array of doubles.
 * @param n The number of elements in each array.
 * @return The dot product of arrays `u` and `v` as a double.
 *
 * @example
 * double dp = dot((double[]){1,1,1,1}, (double[]){1,1,1,1}, 4); // 4.0
 */
auto dot(const double (&u)[], const double (&v)[], const size_t n) {
    auto prod = 0.;
    for (size_t i = 0; i < n; ++i) 
        prod += u[i] * v[i];
    return prod;
}
::::

Note that the input vectors $\M{u}$ and $\M{v}$ are not restricted to the three-dimensional real vector space, i.e., $n=3$.

In [ ]:
%%cpp
auto dot(const double (&u)[], const double (&v)[], const size_t n) {
    auto prod=0.;
    for (size_t i=0; i<n; ++i) prod += u[i]*v[i];
    return prod;
}
cout << dot((double[]){1,0}, (double[]){0,1}, 2) << ' ';
cout << dot((double[]){1,0,0}, (double[]){0,0,1}, 3) << ' ';
cout << dot((double[]){1,0,0,0}, (double[]){0,0,0,1}, 4) << ' ';
cout << dot((double[]){1,0,0,0,0}, (double[]){0,0,0,0,1}, 5) << ' ';
cout << dot((double[]){1,0,0,0,0,0}, (double[]){0,0,0,0,0,1}, 6) << ' ';

The above shows that the dot products of orthogonal vectors in $\mathbb{R}^n$ for different dimensions $n$. However, the dimension needs to be passed into the function as a separate parameter `n`.

::::{exercise}
:label: ex:dot1

Why the declaration `const double (&v)[]` does not work in the following code even though it works in the above function declaration?

```cpp
double v[]{1,2,3};
const double (&u)[] = v;
```

::::

YOUR ANSWER HERE

To implement the norm in [](#eq:norm):

In [ ]:
%%cpp
auto norm(const double (&v)[], const size_t n) {
    return sqrt(dot(v, v, n));
}
cout << norm((double[0]){}, 0) << ' ';
cout << norm((double[1]){1}, 1) << ' ';
cout << norm((double[2]){1,1}, 2) << ' ';
cout << norm((double[3]){1,1,1}, 3) << ' ';
cout << norm((double[4]){1,1,1,1}, 4) << ' ';

This calculates the norm of the vectors of all ones in $\mathbb{R}^n$ for different dimensions $n$.

::::{exercise} 
:label: ex:dot2

Why can't we automate the calculation using the following `for` loop?

```cpp
for (auto n=0u; n<5; n++) {
    double v[n];
    for (auto i=0u; i<n; ++i) v[i]=1;
    cout << norm(v) << ' ';
}
```

::::

YOUR ANSWER HERE

We will learn to use the C++11 container [`std::vector`](https://en.cppreference.com/w/cpp/container/vector.html) from `<vector>`, whose length can be changed dynamically.

## Compound Literal

**How to enter an array of elements?**

Recall that a string is a constant array of characters:

In [ ]:
%%cpp
"abc"

The size is fixed and can be obtained at compile time:

In [ ]:
%%cpp
sizeof("Ava")  // in bytes

Instead of a [string literal](https://en.cppreference.com/w/c/language/string_literal.html), we can enter a string as an array using the C-style [compound literal](https://en.cppreference.com/w/c/language/compound_literal.html):

In [ ]:
%%cpp
cout << (const char[]){'A','v','a', 0};  // 😈: Oh, there is a typo... `, 0`.

We can also specify a bigger size for an array than the size of the initializer:

In [ ]:
%%cpp
cout << (const char[4]){'A','v','a'}; // remaining entries initialized to 0

The entries of an array need not be characters but they must be of the same type:

In [ ]:
%%cpp
(int[]){1,2,3}

In [ ]:
%%cpp
sizeof((double[]){1,2,3})

In [ ]:
%%cpp
sizeof((int[6]){1,2,3})  // `double` is not double the size or `int`?

This is unlike Python `list` and `tuple`, which can be heterogeneous, i.e., with elements of different types:

In [ ]:
[(1, '2'), [False, 3]]

It is more like `array` from `numpy`, which must be homogenous, i.e., with elements of the same types:

In [ ]:
import numpy as np

np.array([(1, '2'), [False, 3]])

An array in C++ can also be multi-dimensional, i.e., an array of arrays of arrays...:

In [ ]:
%%cpp
(int[][2]){{1,2},{3, 4}}  // 2D array

In [ ]:
%%cpp
sizeof((int[][2]){{1,2},{3, 4}})

In [ ]:
%%cpp
(int[][2][3]){{{1,2,3},{4,5,6}},{{7,8,9}, {10,11,12}}}  // 3D array

The memory for a compound literal is statically allocated, and so the length must be fixed at compile time:

In [ ]:
%%cpp
constexpr auto d=3u;
(int[d]){1,2,3}   // works: `d` is a compile-time constant

In [ ]:
%%cpp
const auto d=3u;  // also a compile-time constant due to compiler optimization
(int[][d]){{1,2},{3, 4}}

In [ ]:
%%cpp
auto d=3u;          // mutable
// (int[d]){1,2,3}; // fails: `d` is not a constant
d

In [ ]:
%%cpp
const unsigned d=1+rand()%3;
// (int[d]){};      // fails: `d` is not known at compile time

## Array Declaration

**How to declare an array variable?**

Consider using `auto` to declare an array:

In [ ]:
%%cpp
auto s = (const char[4]){'a','b','c'};
s[0]  // a?

While a variable can normally be initalized with a literal of the same type, a variable cannot be properly initialized with a compound literal.

::::{caution} Why not initialize a variable with compound literal?
:class: dropdown

Compound literals are short-lived and therefore not suitable to be assigned to a variable. E.g., the above code fails to give `s[0]` because a compound literal is destroyed at the end of the full-expression.

::::

This is unlike the [initialization/assignment from string literal](https://en.cppreference.com/w/c/language/array_initialization.html#Initialization_from_strings):

In [ ]:
%%cpp
auto s = "abc";
s

In [ ]:
%%cpp
s = "def";
s

To [declare an array](https://en.cppreference.com/w/cpp/language/array) of `double`, simply add the square brackets `[]` after the variable name to designate the *size* of the array:

In [ ]:
%%cpp
int v[3];
v

Since `v` has a static storage duration, its elements are initialized to `0.0` by default.

An array can also be initialized by a form of [list initialization](https://en.cppreference.com/w/cpp/language/list_initialization.html) called [aggregate initialization](https://en.cppreference.com/w/cpp/language/aggregate_initialization.html):

In [ ]:
%%cpp
int u[3]{1}, v[]{0,1,0}, (&w)[3]=u;
// int v[];      // fails: needs an explicit size or an initializer
// int w[3]=u;   // fails: array initializer must be an initializer list. Why?
v

In the above declaration:
- `u[3]{1}` initializes the first element as `1`, and the subsequent elements are initialized as `0` by default.
- `v[]` is an array of unknown bound. Such an array can be initialized with a non-empty list initializer such as `{1,0,0}`, from which the compiler can deduce how much space to allocate.

::::{exercise}
:label: ex:array-declaration

Explain why each of the following declarations fail.

```cpp
int v[];
int w[3]=u;
```

::::

YOUR ANSWER HERE

## Array as Pointer

After an array has been declared, the same brackets `[]` are used as the [*member access operator*](https://en.cppreference.com/w/cpp/language/operator_member_access.html) to access or modify individual elements. E.g., the following function prints a 3D vector represented as an array of three elements:

::::{code} cpp
:label: code_print_vector3_
:caption: Print a 3D vector represented as an array of three elements.
:linenos:
void print_vector3_(const auto v[3]) {
    cout << format("[{}, {}, {}]\n", v[0], v[1], v[2]);
}
::::

In [ ]:
%%cpp
void print_vector3_(const auto v[3]) {
    cout << format("[{}, {}, {}]\n", v[0], v[1], v[2]);
}
print_vector3_((int[]){1,2,3});

::::{exercise}
:label: ex:print_vector31

Explain why the following two cells give different results?

::::

In [ ]:
%%cpp
{
    int u[3];
    print_vector3_(u);
}

In [ ]:
%%cpp
{
    int u[3] {};
    print_vector3_(u);
}

YOUR ANSWER HERE

The definition of [`print_vector3_`](#code_print_vector3_) is unsafe because it can cause [buffer overrun](https://en.wikipedia.org/wiki/Buffer_overflow):

In [ ]:
%%cpp
print_vector3_((int[]){1,2});   // 😈: What is that last element...

Even though the input array only has two elements, `print_vector3_` prints the element after the second one, accessing an out-of-bound memory location!

```cpp
void print_vector3_(const auto v[3]) {
    ... format(... v[2]);
}
```

If the input array is passed by reference instead, the size is enforced:

::::{code} cpp
:label: code_print_vector3
:caption: Input array of three elements passed by reference.
:linenos:
void print_vector3(const auto (&v)[3]) {  // pass by constant reference
    cout << format("[{}, {}, {}]\n", v[0], v[1], v[2]);
}
::::

In [ ]:
%%cpp
void print_vector3(const auto (&v)[3]) {
    cout << format("[{}, {}, {}]\n", v[0], v[1], v[2]);
}
print_vector3((int[]){1,2,3});
// print_vector3((int[]){1,2});  // fails since double[3] is expected.

Why pass by reference works but pass by value doesn't? We need to understand the implementation and limitation of arrays:

::::{caution}

In C++, an array fails all the requirements for a [first-class citizen](https://en.wikipedia.org/wiki/First-class_citizen).

- An array cannot be assigned to another array.
- An array cannot be passed into a function as a argument.
- An array cannot be returned by a function.

::::

### 1D Array

[An array can be implicitly converted to a pointer](https://en.cppreference.com/w/cpp/language/implicit_conversion.html#Array-to-pointer_conversion) that references the first element of the array:

In [ ]:
%%cpp
int u[]{1,2,3};
*u  // first element

Subsequent elements are stored in the subsequent locations:

In [ ]:
%%cpp
*(u+1)  // second element

In [ ]:
%%cpp
*(u+2)  // third element

The array can therefore be assigned to a pointer or to another array by lvalue reference:

In [ ]:
%%cpp
int u[]{1,2,3};
auto *v=u, (&w)[3]=u;
print_vector3_(v);
print_vector3(w);

::::{exercise}
:label: ex:pointer-vs-array

In the following code, why `u` and `v` have different sizes even though they are equal?

::::

In [ ]:
%%cpp
cout << format("sizeof(u): {}\n", sizeof(u));
cout << format("sizeof(v): {}\n", sizeof(v));
(u==v)

::::{solution} ex:pointer-vs-array
:class: dropdown

`sizeof` calculates the size of the array `u` at compile time because its length is known. The size is 12 because the array contains 3 `int`, each of size 4. 

However, when `u` is assigned to a pointer `v`, the size information is lost — sizeof(v) only returns the size of the pointer itself (typically 8 bytes on a 64-bit system), not the size of the array it points to.

::::

::::{note}

In other words, the subscript operation `v[i]` is equivalent to `*(v+i)` since elements of an array are stored in consecutive memory locations. What a *clever* trick! This implements an array efficiently without any overhead. Even the size of the array, which must be a compile-time constant, needs not be stored as meta data in the executable!

::::

In [ ]:
%%ai
Explain briefly how memory is statically allocated to an array in C++, and 
whether the size of the array needs to be stored in the executable?

While an array can be assigned to a pointer, it cannot be assigned to an array:

```cpp
double u[3]={1,2,3}, v[3];
v = u; // fails: array type 'int[3]' is not assignable
```

In [ ]:
%%ai
Explain briefly why a statically allocated array is not assignable?

When the array argument is passed by reference, the function signature is `(void (*)(int (&)[3]))` as expected.

In [ ]:
%%cpp
void bar1(int (&v)[3]) {}
bar1

However, when the argument is passed by value, the function type is `(void (*)(int *))`:

In [ ]:
%%cpp
void bar2(int v[3]) {}
bar2

In other words, an array cannot be passed by value without decaying to a pointer. In particular, the size information is lost as the parameter decays from `int v[3]` to a pointer `int *`.

We can now answer the question:

::::{important} The size of an input array is enforced when the array is passed by reference instead of value. Why?
:class: dropdown

1. Even though the parameter of [`print_vector3_`](#code_print_vector3_) is declared as an array `auto v[3]`, it decays to a pointer `auto *v` so that the address of the first element of the array can be assigned to the parameter as a pointer. However, the size information is lost in doing so. In other words, the parameter is effectively declared with an unknown bound (`auto v[]`), which allows the compiler to happily accept an array of any size. The function does not know the array’s length, potentially leading to a buffer overrun. 

2. When the parameter is passed by reference (`double (&v)[3]`) to [`print_vector3`](#code_print_vector3), however, the compiler does not treat it as `double *v`, and so the size information is retained.

::::

### Multi-Dimensional Array

A multi-dimensional array can also decays to a pointer in the same way. Consider the following 2-by-2-by-3 array:

In [ ]:
%%cpp
int v[][2][3]{
    // v[0] or *v
    {
        // v[0][0] or **v
        {
            // v[0][0][0] or *(**v + 0)
            1,
            // v[0][0][1] or *(**v + 1)
            2,
            // v[0][0][2] or *(**v + 2)
            3
        },
        // v[0][1] or *(*v + 1)
        {
            // v[0][1][0] or *(**v + 3)
            4,
            // v[0][1][1] or *(**v + 4)
            5,
            // v[0][1][2] or *(**v + 5)
            6
        }
    },
    // v[1] or *(v + 1)
    {
        // v[1][0] or *(*v + 2)
        {
            // v[1][0][0] or *(**v + 6)
            7,
            // v[1][0][1] or *(**v + 7)
            8,
            // v[1][0][2] or *(**v + 8)
            9
        },
        // v[1][1] or *(*v + 3)
        {
            // v[1][1][0] or *(**v + 9)
            10,
            // v[1][1][1] or *(**v + 10)
            11,
            // v[1][1][2] or *(**v + 11)
            12
        }
    }
};
v

::::{caution} Why the bound of the first (instead of the last) dimension of a multi-dimensional array can be unknown?
:class: dropdown

Think of a multi-dimensional array as an array, whose element happens to be arrays themselves. The bound of the dimension, namely the first dimension, can be unknown as in `int v[][2][3]` if there is an initializer.

::::

::::{exercise} 
:label: ex:fixed-size

Explain why the following code fails even though `d` is a constant array?

```cpp
const int d[] {1,2};
(int[][d[0]][d[1]]){{{1,2},{3, 4}}};
```

::::

YOUR ANSWER HERE

It can also be passed into a function with a parameter declared as `int (&v)[2][2][3]`, `int v[][2][3]`, or `int (*v)[][3]`.

In [ ]:
%%cpp
void foo1(int (&v)[2][2][3]) {}
foo1(v);
foo1

In [ ]:
%%cpp
void foo2(int v[][2][3]) {}
foo2(v);
foo2

In [ ]:
%%cpp
void foo3(int (*v)[2][3]) {}
foo3(v);
static_cast<int (*)[2][3]>(v);
foo3

::::{exercise}
:label: ex:pointer-of-pointers

Why does the following code fail?

```cpp
void foo4(int (**v)[]) {};
foo4(v);                         // fails
static_cast<int (**)[][3]>(v);  // fails
```

::::

YOUR ANSWER HERE

The 3D array `v` is actually stored like a 1D array `**v` in memory as shown below:

In [ ]:
%%cpp
print_vector3(**v);
print_vector3(*(*v+1));
print_vector3(*(*v+2));
print_vector3(*(*v+3));

To print all elements in one go:

In [ ]:
%%cpp
for (auto l=0u; l<2*2*3; l++)
    cout << (**v)[l] << ' ';  // `(**v)[l]` is the same as `(&v[0][0][0])[l]`

`**v` is the same as `&v[0][0][0]`, i.e., the address to the first element of the 3D array.

This is equivalent to printing each element of the 3D array using nested for loops:

In [ ]:
%%cpp
constexpr unsigned d[]{2,2,3};
for (auto i=0u; i<d[0]; i++)
    for (auto j=0u; j<d[1]; j++)
        for (auto k=0u; k<d[2]; k++)
            cout << v[i][j][k] << ' ';

`v[i][j][k]` is the same as `*(**v+i*(d[1]*d[2])+j*(d[2])+k)`:

In [ ]:
%%cpp
for (auto i=0u; i<d[0]; i++)
    for (auto j=0u; j<d[1]; j++)
        for (auto k=0u; k<d[2]; k++)
            cout << *(**v+i*(d[1]*d[2])+j*(d[2])+k) << ' ';

::::{seealso} C-ordering

The above prints each element of the array (`v`) by incrementing the indices (`i`, `j`, and `k`) of each dimension circularly, with the index of an earlier dimension (known as a row index) incremented slower. This is indeed how the elements are stored linearly in memory in C/C++, and so the ordering is known as C-ordering or row-major ordering.[^F-ordering]

[^F-ordering]: In comparison, Fortran uses a column-major ordering or F-ordering, where the index of a later dimension (known as a column index) incremented slower.

## Array Container

[](#code_cross_cstyle) implements the cross product using C-style fixed array:

```cpp
void cross(..., ..., double (&w)[3]) {
    w[0] = ...
    w[1] = ...
    w[2] = ...
}
```

Isn't it strange that the function takes the cross product `w` as an input argument instead of returning it as an output?

To understand the issue, consider a simplified function that returns an array:

In [ ]:
%%cpp
auto foo() {
    double w[1]{123};
    cout << *w << " @" << static_cast<void *>(w) << '\n';
    return w;
}
auto w=foo()

The return type is `double *` instead of `double [1]`, which means that an array cannot be returned by a function without decaying to a pointer.

Note however that there is a warning about the `address of stack memory ... returned`. To see the value it points to:

In [ ]:
%%cpp
cout << *w << " @" << static_cast<void *>(w) << '\n';
w[0] // equal 1?

Observe that `static_cast<void *>(w)` remains the same but `*w` has changed probably. Why?

::::{caution}

Function parameters or variables in a block scope are stored in the so-called stack memory (or simply the stack). They are only valid while the scope is active, i.e., they are cleaned up automatically when they fall out of scope. They are said to have an [automatic storage duration](https://en.cppreference.com/w/cpp/language/storage_duration.html#Automatic_storage_duration).

::::

In [ ]:
%%ai
Explain briefly what stack memory is in C++ and why returning address of stack
memory gives a warning?

As a comparison, consider the following function, which returns a locally defined `double`?

In [ ]:
%%cpp
auto foo() {
    double w=123.;
    return w;
}
auto w=foo()

::::{caution} Why the above code does not trigger any warning even though the variable `w` has automatic storage duration?
:class: dropdown

Due to the [return value optimization](https://en.cppreference.com/w/cpp/language/copy_elision.html), the result returned by a function is constructed directly into the function call's result object, which is therefore in the scope where the function is called. However, if `w` is a pointer to an array, it is the pointer `w` itself—not the array it points to—that is constructed into the caller's result object. This explains why the address `static_cast<void *>(w)` remains the same but `*w` has changed.

::::

One way to return an array is to declare it as static:

In [ ]:
%%cpp
auto foo() {
    static const double w[1]{123};
    return w;
}
auto w=foo();
w[0]

The memory location `w[0]` has static storage duration, and therefore remains available throughout the execution of the program. However, the function is not thread-safe unless `w` is declared to be constant.

**How to return a mutable array safely?**

In [ ]:
%%cpp
auto foo() {
    auto w=new double[1]{123};
    return w;
}
auto w=foo();
++w[0]

The above function returns a mutable array by dynamically allocating memory on the heap memory using the [`new` expression](https://en.cppreference.com/w/cpp/language/new.html). It is thread-safe because every call to the function creates a distinct heap-allocated array, and so there is no shared state between threads. The heap memory remains accessible after the function call because it is not tied to the function's stack frame.

::::{caution} [Memory leaks](https://en.wikipedia.org/wiki/Memory_leak)

Imagine what happens if `foo` is repeatedly called like this:

```cpp
while (true) foo();   //  😈: A small devil with a big appetite!

```

::::

Since every function call creates a new array, and the memory is not released after the call completes, the caller is responsible for freeing the memory with `delete[]`.

In [ ]:
%%cpp
delete[] w;
// w           // fails as the memory for w is released.
// delete[] w; // fails for the same reason.
w=nullptr;     // prevents accidental reuse or double deletion.

::::{exercise}
:label: ex:zeros

Explain all the issues with the following code:

```cpp
double* zeros(const int d) {
    double v[d]{};
    return v;
}

double x[5]= zeros(5);
```

::::

YOUR ANSWER HERE

Thanks to object-oriented programming, `C++11` defines a value-type container [`std::array`](https://en.cppreference.com/w/cpp/container/array.html) from `<array>`:

In [ ]:
%%cpp
auto foo() {
    auto w=array<double, 1>{123};
    return w;
}
auto w=foo();
++w[0];
w

This works as if `w` were a simple data type:

- The array object is correctly returned because it is directly constructed into the caller’s result object via the return value optimization.
- The function is thread-safe since each call creates a distinct array object.
- The memory is released automatically when the result object goes out of scope.

We can now modify [](#code_cross_cstyle) to return the cross product:

::::{code} cpp
:label: code_cross
:caption: Compute the cross product $\M{u}\times \M{v}$ using [`std::array`][array] from `<array>`.
:linenos:
/**
 * @brief Computes the cross product of two 3D vectors.
 *
 * This function calculates the cross product of two 3-dimensional (3D) vectors `u` and `v`,
 * both represented as `std::array<double, 3>`. The result is a new vector that is
 * perpendicular to both `u` and `v`, following the right-hand rule.
 *
 * @param u The first input vector.
 * @param v The second input vector.
 * @return A 3D vector representing the cross product of `u` and `v`.
 * @example
 * auto w = cross({1,0,0}, {0,0,1}); // (vector3D &) { 0.0, -1.0, 0.0 }
 */
using vector3d = std::array<double, 3>;    // type alias

vector3d cross(const vector3d &u, const vector3d &v) {
    vector3d w;
    w[0] = u[1]*v[2] - u[2]*v[1];
    w[1] = u[2]*v[0] - u[0]*v[2];
    w[2] = u[0]*v[1] - u[1]*v[0];
    return w;
}
::::

[array]: https://en.cppreference.com/w/cpp/container/array.html

For convenience, the code defines a [type alias](https://en.cppreference.com/w/cpp/language/type_alias.html) `vector3d` for the type `std::array<double, 3>`:

In [ ]:
%%cpp
using vector3d=std::array<double, 3>;

The type alias can then be used to declare the parameters and return value:

In [ ]:
%%cpp
vector3d cross(const vector3d &u, const vector3d &v) {
    vector3d w;
    w[0] = u[1]*v[2] - u[2]*v[1];
    w[1] = u[2]*v[0] - u[0]*v[2];
    w[2] = u[0]*v[1] - u[1]*v[0];
    return w;
}

auto w = cross({1,0,0}, {0,0,1});
w

The array container `w` is defined within the function and returned to the caller, just like other simple data types:

```cpp
vector3d cross(..., ...) {
    vector3d w;
    w[0] = ...
    w[1] = ...
    w[2] = ...
    return w;
}
```

::::{caution}

`u` and `v` are passed by constant reference to avoid the extra copy step involved if the arguments were passed by value instead. `const` allows rvalue to be passed by lvalue reference.

::::

As another example demonstrating the benefit of array container, the following code implements the [wheel factorization](https://en.wikipedia.org/wiki/Wheel_factorization) to check whether a number `n` is prime or not:

::::{code} cpp
:label: code_is_prime
:caption: Check whether a non-negative integer is prime.
:linenos:
/**
 * @brief Determines whether a given unsigned long long integer is prime using wheel factorization.
 *
 * This function checks for primality by first testing divisibility against a small set of base primes {2, 3, 5}.
 * It then uses wheel factorization based on the LCM of these primes (30) to efficiently skip non-candidate divisors.
 * Candidate divisors are generated by adding coprime offsets to multiples of 30.
 *
 * @param n The number to check for primality.
 * @return true if n is prime, false otherwise.
 *
 * @details
 * - The wheel is constructed using the coprimes of 30: {1, 7, 11, 13, 17, 19, 23, 29}.
 * - These offsets are added to each multiple of 30 to generate candidate divisors.
 * - The function stops checking once the candidate divisor exceeds sqrt(n).
 *
 * @note This method avoids redundant checks for multiples of 2, 3, and 5, improving efficiency for large numbers.
 *
 * @example
 *     bool result = is_prime(97); // returns true
 */
bool is_prime(const unsigned long long n) {
    if (n < 2) return false;

    // Base primes used for initial divisibility check
    auto basis = to_array<unsigned long long>({2, 3, 5});

    // Offsets that are coprime to 30 (LCM of 2, 3, 5)
    auto offsets = to_array<unsigned long long>({1, 7, 11, 13, 17, 19, 23, 29});

    const unsigned long long m = 30; // LCM of base primes
    const unsigned long long limit = sqrt(n); // No need to check beyond sqrt(n)

    // Check divisibility by base primes
    for (auto p : basis)
        if (p > limit) return true; // No need to check further
        else if (n % p == 0) return false;

    // Use wheel factorization to check remaining candidates
    unsigned long long d;
    for (unsigned long long i = 0uLL;; i += m) {
        for (auto j : offsets) {
            d = i + j;
            if (d > limit) return true; // No divisor found up to sqrt(n)
            else if (n % d == 0 && d > 1) return false; // Found a divisor
        }
    }
}

::::

In [ ]:
%%cpp
bool is_prime(const unsigned long long n) {
    if (n<2) return false;
    auto basis=to_array<unsigned long long>({2,3,5});
    auto offsets=to_array<unsigned long long>({1, 7, 11, 13, 17, 19, 23, 29});
    const unsigned long long m=30, limit=sqrt(n);
    for (auto p : basis)
        if (p > limit) return true; else
        if (n % p == 0) return false;
    unsigned long long d;
    for (unsigned long long i=0uLL;; i+=m)
        for (auto j : offsets)
            if ((d=(i+j))>limit) return true; else
            if (n%d==0 && d>1) return false;
}

In [ ]:
%%cpp
for (auto n: views::iota(0, 100)) if (is_prime(n)) cout << n << ' ';

The above code used the following techniques:
- The array container supports [`range-based` `for` loop](https://en.cppreference.com/w/cpp/language/range-for.html) to iterate over its elements simply without using indices.
- [`std::to_array`](https://en.cppreference.com/w/cpp/container/array/to_array.html) introduced in C++20 allows the length and data type to be deduced from the manually specified array.

## Vector Container

Recall that the declaration `const auto v[3]` in [`void print_vector3_(const auto v[3])`](#code_print_vector3_) is the same as `const auto v[]`, which decays to `const auto *v`. Since the size information is lost, we might as well define a more generate function below to print an arbitrarily long vector:

::::{code} cpp
:label: code_print_vector_cstyle
:caption: Print the contents of a C-style array.
:linenos:
/**
 * @brief Prints the contents of a vector-like array to standard output in formatted style.
 *
 * This function takes an array of elements and its size, then prints the elements
 * in a comma-separated list enclosed in square brackets. The formatting uses `std::format`
 * for consistent output.
 *
 * @tparam T Type of the elements in the array (deduced automatically).
 * @param v Array of elements to print.
 * @param n Number of elements in the array.
 *
 * @note If the array is empty (`n == 0`), nothing is printed.
 * @example
 *     int arr[] = {1, 2, 3};
 *     print_vector(arr, 3); // Output: [1, 2, 3]
 */
void print_vector(const auto v[], const size_t n) {
    if (n) {
        cout << format("[{}", v[0]);
        for (size_t i = 1; i < n; ++i)
            cout << format(", {}", v[i]);
        cout << "]\n";
    }
::::

In [ ]:
%%cpp
void print_vector(const auto v[], const size_t n) {
    if (n) {
        cout << format("[{}", v[0]);
        for (size_t i=1; i<n; ++i) cout << format(", {}", v[i]);
        cout << "]\n";
    }
}
print_vector((int[]){1}, 1);
print_vector((int[]){1, 2}, 2);
print_vector((int[]){1, 2, 3}, 3);
print_vector((int[]){1, 2, 3, 4}, 4);
print_vector((int[]){1, 2, 3, 4, 5}, 5);

In the function calls, the size is passed as a separate argument `n` because the size information in `v` is lost in as `v` decays to a pointer.

To print `std::array` instead:

::::{code} cpp
:label: code_print_vector_array
:caption: Print the contents of an array container.
:linenos:
/**
 * @brief Prints the contents of a fixed-size std::array in vector format.
 *
 * This function outputs the elements of the array in the format: [x1, x2, ..., xN].
 * It uses C++11's std::array from <array> and std::format for formatting.
 *
 * @tparam T Type of the elements in the array.
 * @tparam N Size of the array.
 * @param v The std::array to be printed.
 * @example
 * print_vector(array<int, 4>{1, 2, 3, 4});
 * print_vector(array<int, 5>{1, 2, 3, 4, 5});
 */
template <class T, size_t N>
void print_vector(const array<T, N> &v) {
    if (N>0) {
        auto it=v.begin();
        cout << format("[{}", *it++);
        for (;it != v.end(); it++) cout << format(", {}", *it);
        cout << "]\n";
    }
}

::::

In [ ]:
%%cpp
template <class T, size_t N>
void print_vector(const array<T, N> &v) {
    if (N>0) {
        auto it=v.begin();
        cout << format("[{}", *it++);
        for (;it != v.end(); it++) cout << format(", {}", *it);
        cout << "]\n";
    }
}
print_vector(to_array({1}));
print_vector(to_array({1, 2}));
print_vector(to_array({1, 2, 3}));
print_vector(to_array({1, 2, 3, 4}));
print_vector(to_array({1, 2, 3, 4, 5}));

::::{caution}

While `print_vector3({1,2,3})` is allowed, `print_vector({1,2,3})` fails because the type and class cannot be inferred directly from the initializer list. `to_array` fixes the issue.

::::

The above code is a [template declaration](https://en.cppreference.com/w/cpp/language/templates.html) which allows the function `print_vector` to take an `std::array<T, N>` with any data type `T` and fixed size `N`. It also uses [iterators](https://en.cppreference.com/w/cpp/container/array.html#Iterators) to traverse the array.

In [ ]:
%%ai
Explain briefly how to use iterators to traverse an array efficiently in C++.

::::{exercise}
:label: ex:array-size

What is the benefit of using template instead of declaring the argument as `auto`?

```cpp
void print_vector(const auto &v) {
    if (!v.empty()) {
        cout << format("[{}", v[0]);
        for (auto vi : v) cout << format(", {}", vi);
        cout << "]\n";
    }
}
```

[`empty`](https://en.cppreference.com/w/cpp/container/array/empty.html) is a method to check whether an array is empty.

::::

YOUR ANSWER HERE

YOUR ANSWER HERE

A common strategy is to set the size to a large enough constant:

In [ ]:
%%cpp
constexpr auto N=5u;
int v[N];
for (auto i=0u; i<N; ++i) v[i]=i+1;
for (auto n=1u; n<=N; ++n) print_vector(v, n);

But what if we do not know how large is large enough at compile time? An alternative is to dynamically allocate the memory to the heap:

In [ ]:
%%cpp
for (auto n=1u; n<=5; ++n) {
    auto v=new int[5];
    for (auto i=0u; i<n; ++i) v[i]=i+1;
    print_vector(v, n);
    delete[] v;   // 😈: This line can be removed. 👨🏻‍🏫: No.
}

To avoid memory leak, C++11 offers variable-length containers such as [`std::vector`](https://en.cppreference.com/w/cpp/container/vector.html) from `<vector>`. This is used in the following implementation of `print_vector`:

In [ ]:
%%cpp
template <class T>
void print_vector(const vector<T> &v) {
    if (!v.empty()) {
        cout << format("[{}", v[0]);
        for (auto vi: v) cout << format(", {}", vi); // range-based for loop
        cout << "]\n";
    }
}

The `print_vector` function used the method [`empty`](https://en.cppreference.com/w/cpp/container/vector/empty.html) to check whether `v` is an empty vector.

::::{important}

For `std::array<T, N>`, the size `N` is required at compile time and the array is allocated on the stack, not the heap. In contrast, `std::vector<T, Allocator>` does not require a fixed size at declaration. Instead, it uses an allocator such as the default [`std::allocator`](https://en.cppreference.com/w/cpp/memory/allocator.html) to allocate heap memory.

- The [`vector` constructors](https://en.cppreference.com/w/cpp/container/vector/vector.html) uses the allocator to allocate memory dynamically.
- The [`vector` destructor](https://en.cppreference.com/w/cpp/container/vector/~vector.html) uses the allocator to release memory when the vector goes out of scope.

::::

We can now use a `for` loop to modify the length of a vector:

```cpp
{
    vector<double> v; /* Creates an empty vector using allocator<double> internally.
                         No memory is allocated yet; capacity is 0. */

    for (auto n = 1u; n <= 5; ++n) { // Loop to insert values 1 to 5
        v.push_back(n); /* Adds 'n' to the vector.
                           If current capacity is full, vector allocates more memory:
                           - Calls allocator.allocate(new_capacity)
                           - Moves existing elements to new memory
                           - Calls allocator.deallocate(old_memory)
                           - Constructs new element with allocator.construct() */
        print_vector(v);
    }
}
/* When 'v' goes out of scope at the end of main:
   - vector's destructor is called
   - Calls allocator.destroy() for each element
   - Calls allocator.deallocate() to free the memory block */
```

In [ ]:
%%cpp
{
    vector<double> v;
    for (auto n=1u; n<=5; ++n) {
        v.push_back(n);
        print_vector(v);
    }
}

In [ ]:
%%ai
Explain briefly what allocators are in C++ and the benefits of using allocators
instead of new and delete in C++?

The code used the method [`push_back`](https://en.cppreference.com/w/cpp/container/vector/push_back.html) to append an element to the end of the vector.

The dot product in [](#code_dot_cstyle) can also be reimplemented to use the vector container:

::::{code} cpp
:label: code_dot
:caption: Compute the dot product $\M{u}\cdot \M{v}$ using [`std::vector`](https://en.cppreference.com/w/cpp/container/vector.html) from `<vector>`.
:linenos:
using vectord = std::vector<double>;

/**
 * @brief Computes the dot product of two vectors of type vectord.
 *
 * This function calculates the dot product of two vectors `u` and `v`.
 * It throws a runtime error if the vectors are not of the same length.
 *
 * @param u A vector of doubles.
 * @param v A vector of doubles.
 * @return The dot product of vectors `u` and `v` as a double.
 *
 * @throws std::runtime_error if the input vectors have different lengths.
 *
 * @example
 * dot({1,1,1,1}, {1,1,1,1}); // 4.0
 */
auto dot(const vectord &u, const vectord &v) {
    if (u.size() != v.size()) 
        throw std::runtime_error("u and v cannot have different lengths.");
    auto prod = 0.;
    for (size_t i = 0, n = u.size(); i < n; ++i) 
        prod += u[i] * v[i];
    return prod;
}
::::

In [ ]:
%%cpp
using vectord=std::vector<double>;

In [ ]:
%%cpp
auto dot(const vectord &u, const vectord &v) {
    if (u.size()!=v.size()) 
        throw runtime_error("u and v cannot have different lengths.");
    auto prod=0.;
    for (size_t i=0, n=u.size(); i<n; ++i) prod += u[i]*v[i];
    return prod;
}
// dot({1,1,1}, {1,1,1,1});  // fails: size mismatch
dot({1,1,1,1}, {1,1,1,1})

The code ensures the input values `u` and `v` are valid by checking whether they have the same size using the method [`size`](https://en.cppreference.com/w/cpp/container/vector/size.html).

To repeatedly calculate the norms of vectors of all ones for different dimensions:

In [ ]:
%%cpp
auto norm(const vectord &v) {
    return sqrt(dot(v, v));
}
vectord v;
for (auto n=0u; n<10; ++n) {
    cout << norm(v) << ' ';
    v.push_back(1);
}

To repeatly calculate the dot products of orthogonal vectors for different dimensions:

In [ ]:
%%cpp
vectord u{0}, v{0};
for (auto n=0u; n<10; ++n) {
    u.insert(u.begin(),1);
    v.push_back(1);
    cout << dot(u, v) << ' ';
    u[0]=v[v.size()]=0;
}

The code used the method [`insert`](https://en.cppreference.com/w/cpp/container/vector/insert.html) to insert `1` to the beginning position specified by the method [`begin`](https://en.cppreference.com/w/cpp/container/vector/begin.html).

::::{exercise}

Fix the program in [](#ex:zeros) to return a `std::vector` of `d` zeros of type `double`.

:::{hint}

Use the [fourth constructor](https://en.cppreference.com/w/cpp/container/vector/vector.html) for `std::vector`.

:::

::::

In [ ]:
%%cpp
auto zeros(const int d) {
    /*
    # REPLACE THE ENTIRE COMMENT WITH YOUR CODE #
    */
}
// test
zeros(11)

::::{exercise}

Complete the following function to implement the [](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes) to return a vector container of all prime numbers less than or equal to the argument `n`.

::::

In [ ]:
%%cpp
/**
 * @brief Generates all prime numbers less than or equal to a given number.
 *
 * This function uses the Sieve of Eratosthenes algorithm to find all prime numbers
 * up to and including the input value `n`. It returns a std::vector of the primes.
 *
 * @param n The upper bound (inclusive) for prime number generation.
 * @return std::vector<unsigned long long> A list of prime numbers ≤ n.
 */
std::vector<unsigned long long> prime_below(unsigned long long n) {
    std::vector<unsigned long long> prime_numbers;
    std::vector<bool> is_prime(n, true);
    /*
    # REPLACE THE ENTIRE COMMENT WITH YOUR CODE #
    */
}
prime_below(100)

In [ ]:
%%cpp
if (prime_below(100000uLL).back() != 99991uLL) throw runtime_error("test failed");